# MongoDB Aggregation Pipeline Demo

![Pipe](pipe.jpg)

In [ ]:
%alias mongo "C:\Program Files\MongoDB\Server\3.4\bin\mongo.exe" Demo --eval %s

### Our collection

```JSON
[
	{
	'name': 'Bob',
	'address': {
		'Street': 'Olive Blvd',
		'City': 'Burbank',
		'State': 'CA'
		},
	'ordered': ['lamb', 'veal', 'chicken'],
	'lifetime_value': 450
	},
	{
	'name': 'Sam',
	'address': {
		'Street': 'Magnolia Blvd',
		'City': 'Burbank',
		'State': 'CA'
		},
	'ordered': ['turkey', 'duck', 'chicken'],
	'lifetime_value': 1000
	},
	{
	'name': 'Jess',
	'address': {
		'Street': 'Hollywood Blvd',
		'City': 'Hollywood',
		'State': 'CA'
		},
	'ordered': ['beef', 'lamb', 'chicken'],
	'lifetime_value': 200
	}
	{
	'name': 'Alejandra',
	'address': {
		'Street': 'Segway Dr',
		'City': 'Bloomington',
		'State': 'IN'
		},
	'ordered': ['shrimp', 'beef'],
	'lifetime_value': 2200
	}
]
```

In [ ]:
mongo "db.customers.insert([{'name':'Bob','address':{'Street':'Olive Blvd','City':'Burbank','State':'CA'},'ordered':['lamb','veal','chicken'],'lifetime_value':450},{'name':'Sam','address':{'Street':'Magnolia Blvd','City':'Burbank','State':'CA'},'ordered':['turkey','duck','chicken'],'lifetime_value':1000},{'name':'Jess','address':{'Street':'Hollywood Blvd','City':'Hollywood','State':'CA'},'ordered':['beef','lamb','chicken'],'lifetime_value':200},{'name':'Alejandra','address':{'Street':'Segway Dr','City':'Bloomington','State':'IN'},'ordered':['shrimp','beef'],'lifetime_value':2200}])"

### How many of our customers are from California?

In [ ]:
mongo "db.customers.aggregate([{$match:{'address.State': 'CA'}}])"

In [ ]:
mongo "db.customers.aggregate([{$match:{'address.State':'CA'}},{$count:'californians'}])"

*Mongoose*
```javascript
app.get('/match', function(req, res){
	User.aggregate(
		{$match: {'address.State': 'CA'}},
		{$count: 'californians'}
	, function(err, users){
		if(err){
			console.log(err);
		} else {
			console.log(users);
			return res.json(users);
		}
	});
});
```

### How profitable is Burbank?

#### Group first, then match.

In [ ]:
mongo "db.customers.aggregate([{$group:{ _id: '$address.City', total_value: {$sum: '$lifetime_value'}}}])"

In [ ]:
*Mongoose*
```javascript

```

In [ ]:
mongo "db.customers.aggregate([{$group:{ _id: '$address.City', total_value: {$sum: '$lifetime_value'}}}, {$match: {_id: 'Burbank'}}])"

*Mongoose*
```javascript
app.get('/group', function(req, res){
	User.aggregate([
		{$group: {
			_id: '$address.City',
			total_value: {$sum: '$lifetime_value'}
		}}]
	, function(err, users){
		if(err){
			console.log(err);
		} else {
			console.log(users);
			return res.json(users);
		}
	})
});
```

#### Match first, then group

In [ ]:
mongo "db.customers.aggregate([{$match: { 'address.City': 'Burbank'}}])"

In [ ]:
mongo "db.customers.aggregate([{$match: { 'address.City': 'Burbank'}}, {$group: {_id: 'address.City', total_value: {$sum: '$lifetime_value'}}}])"

### Suddenly, we need a seperate collection for the orders.

#### Unwind Magic

In [ ]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}])"

*Mongoose*
```javascript
app.get('/unwind', function(req, res){
	User.aggregate([
		{$unwind: '$ordered'}
	], function(err, ordered){
		if(err){
			console.log(err);
		} else {
			console.log(ordered);
			return res.json(ordered);
		}
	});
});
```

#### Collections need unique \_ids. Using addFields.

In [ ]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}])"

#### Creating a new collection within the aggregate pipeline

In [ ]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}, {$out: 'orders'}])"

In [ ]:
mongo "db.orders.find()"

#### Redundant and Unnecessary fields! Drop the bass...

In [ ]:
mongo "db.orders.drop()"

#### Projecting(similar to second argument in `find()`)

In [ ]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}, {$project: {id: 1, address: 1}}, {$out: 'orders'} ])"

In [ ]:
mongo "db.orders.find()"

*Mongoose*
```javascript
app.get('/out', function(req, res){
	User.aggregate([
		{$unwind: '$ordered'},
		{$addFields: {_id: {name: '$name', ordered: '$ordered'}}},
		{$project: {id: 1, address: 1}},
		{$out: "orders"}
	], function(err, coll){
		if(err){
			console.log(err);
		} else {
			mongoose.connection.db.collection('orders', function(err, collection){
				collection.find({}).toArray(function(err, orders){
					if(err){
						console.log(err);
					} else {
						for(var i = 0; i < orders.length; i++){
							quantity = Math.round(Math.random()*3)+1;
							orders[i].quantity = quantity;
							collection.save(orders[i]);
						}
						return res.json(orders);
					}
				});
			});
		}
	});
});
```

### Suddenly, we recieve quantities(finally)

In [ ]:
mongo "var orders=db.orders.find(); orders.forEach(function(order){order.quantity=Math.round(Math.random()*3)+1;db.orders.save(order)})"

In [ ]:
mongo "db.orders.find().pretty()"

### Lets see which products are bought in greater quantities! AKA "Who loves joins?!"

#### Grouping prior to joining and creating an array of order: quantity key-value pairs.

In [ ]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}])"

#### Join and create new collection 

In [ ]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}, {$lookup: {from: 'customers', localField: '_id', foreignField: 'name', as: 'cust'}}]).pretty()"

In [ ]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}, {$lookup: {from: 'customers', localField: '_id', foreignField: 'name', as: 'cust'}}, {$out: 'cmb'}])"

*Mongoose*
```javascript
app.get('/out1', function(req, res){
	mongoose.connection.db.collection('orders', function(err, collection){
		collection.aggregate([
				{$group: 
					{_id: '$_id.name', 
					ordQuant: {
						$push: {order: '$_id.ordered', quantity: '$quantity'}
					}}},
				{$lookup: {
					from: 'users',
					localField: '_id',
					foreignField: 'name',
					as: 'cust'
				}},
				{$out: 'cmb'}
			]).toArray(function(err, orders){
			if(err){
				console.log(err);
			} else {
				mongoose.connection.db.collection('orders', function(err, collection){
					collection.find({}).toArray(function(err, orders){
						if(err){
							console.log(err);
						} else {
							console.log(orders);
							return res.json(orders);
						}
					});
				});
			}
		});
	});
});
```

#### Unwind the new collection

In [ ]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}])"

#### Groups the products and averages out their quantities 

In [ ]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}, {$group: {_id: '$ordQuant.order', average_quantity: {$avg: '$ordQuant.quantity'}}}])"

# AND FINALLY

#### SORT!(numerical and alphabetical)

In [ ]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}, {$group: {_id: '$ordQuant.order', average_quantity: {$avg: '$ordQuant.quantity'}}}, {$sort: {average_quantity: -1, _id: 1}  }])"

*Mongoose*
```javascript
app.get('/group1', function(req, res){
	mongoose.connection.db.collection('cmb', function(err, collection){
		collection.aggregate([
				{$unwind: '$ordQuant'},
				{$group: {
					_id: '$ordQuant.order',
					average_quantity: {$avg: '$ordQuant.quantity'}
				}},
				{$sort: {average_quantity: -1, _id: 1}}
			]).toArray(function(err, orders){
			if(err){
				console.log(err);
			} else {
				console.log(orders);
				return res.json(orders);
			}
		});
	});	
});
```

In [ ]:
mongo "db.cmb.drop(); db.customers.drop(); db.orders.drop()"

In [ ]:
mongo "db.getCollectionNames()"